In [3]:
import pandas as pd
pd.options.mode.chained_assignment = None
import urllib.request
import tarfile
import os
import cv2
import requests
import numpy as np
import tensorflow as tf
from tqdm import tqdm
from pytube import YouTube

In [4]:
dataset_download_path="./asl_dataset"
MSASL_test = pd.read_json("/Users/archishabhattacharya/Desktop/Test/MS-ASL/MSASL_test.json".format(dataset_download_path))
MSASL_test.head(10)

,org_text,clean_text,start_time,signer_id,signer,start,end,file,label,height,fps,end_time,url,text,box,width,review
0,beer,beer,0.0,20,40,0,59,SignSchool Beer var,805,360,29.970,1.969,www.youtube.com/watch?v=wX78EPtSuzU,beer,"[0.046787232160568, 0.290409207344055, 1.0, 0....",640,NaN
1,enjoy,enjoy,0.0,11,39,0,54,enjoy(1),192,360,29.970,1.802,www.youtube.com/watch?v=OL02Odh2dRg,enjoy,"[0.071450918912887, 0.136024981737136, 1.0, 0....",480,NaN
2,emotional,emotional,0.0,20,40,0,30,SignSchool Emotional,907,360,23.976,1.251,www.youtube.com/watch?v=C59jcSo4fEI,emotional,"[0.059554219245910006, 0.281019657850265, 1.0,...",640,NaN
3,key,key,0.0,92,-1,0,37,SignSchool Key,456,360,23.976,1.543,www.youtube.com/watch?v=Qs2ua1S6tg0,key,"[0.11462894082069301, 0.308987438678741, 1.0, ...",640,NaN
4,bad,bad,0.0,11,39,0,36,BAD,81,360,29.970,1.201,www.youtube.com/watch?v=-kgTBeOw95A,bad,"[0.040453493595123007, 0.22047379612922602, 0....",480,NaN
5,NEED,need,0.0,67,11,0,68,NEED,29,720,29.970,2.269,https://www.youtube.com/watch?v=ieLzL1Sl6SU,need,"[0.013118863105773001, 0.28781014680862405, 1....",1280,NaN
6,build,build,0.0,20,40,0,50,SignSchool Build 2,394,360,29.970,1.668,www.youtube.com/watch?v=xiJYYCuA2UA,build,"[0.013172954320907001, 0.19094446301460202, 0....",640,NaN
7,subway,subway,0.0,92,-1,0,33,SignSchool Subway,886,360,23.976,1.376,www.youtube.com/watch?v=qSOspusLa3U,subway,"[0.09784886240959101, 0.32842993736267, 1.0, 0...",640,NaN
8,pull,pull,0.0,92,43,0,32,SignSchool Pull(1),672,360,23.976,1.335,www.youtube.com/watch?v=78bHeANdyYc,pull,"[0.10073888301849301, 0.19051924347877502, 1.0...",640,NaN
9,happy,happy,0.0,11,39,0,46,HAPPY,5,360,29.970,1.535,www.youtube.com/watch?v=deGERv54kIc,happy,"[0.049813270568847004, 0.20324450731277402, 0....",480,NaN


In [ ]:
# Define constants
IMAGE_HEIGHT, IMAGE_WIDTH, NUM_CLASSES, EPOCHS = 128, 128, 1000, 10

# Load dataset
dataset_download_path = "./asl_dataset"
MSASL_train = pd.read_json("/Users/archishabhattacharya/Desktop/Test/MS-ASL/MSASL_train.json".format(dataset_download_path))
MSASL_test = pd.read_json("/Users/archishabhattacharya/Desktop/Test/MS-ASL/MSASL_test.json".format(dataset_download_path))
MSASL_val = pd.read_json("/Users/archishabhattacharya/Desktop/Test/MS-ASL/MSASL_val.json".format(dataset_download_path))

def video_to_frames(url, start_time, end_time, bbox, width, height):
    frames = []
    temp_file = "temp_video.mp4"

    try:
        # Download the video using pytube
        yt = YouTube(url)
        stream = yt.streams.filter(progressive=True, file_extension="mp4").first()
        stream.download(filename=temp_file)
    except Exception as e:
        print(f"Failed to download video from {url}. Reason: {e}")
        return frames  # Return an empty list for private videos or any other download error

    # Now read with OpenCV from the downloaded temporary file, NOT the URL
    vid = cv2.VideoCapture(temp_file)
    
    if not vid.isOpened():  # Check if video is loaded correctly
        print(f"Failed to read video from {temp_file}")
        return frames
    
    for timepoint in np.arange(start_time, end_time, 1/30):  # Assuming 30fps for sampling
        vid.set(cv2.CAP_PROP_POS_MSEC, timepoint*1000)
        ret, frame = vid.read()
        if ret:
            y0, x0, y1, x1 = [int(coord * dim) for coord, dim in zip(bbox, [height, width, height, width])]
            roi = frame[y0:y1, x0:x1]  # Extract the region of interest
            frames.append(cv2.resize(roi, (IMAGE_WIDTH, IMAGE_HEIGHT)))

    # Cleanup
    vid.release()
    os.remove(temp_file)
    
    return frames

def dataframe_to_dataset(df):
    data = []
    labels = []
    for _, row in tqdm(df.iterrows(), total=len(df)):
        frames = video_to_frames(row['url'], row['start_time'], row['end_time'], row['box'], row['width'], row['height'])
        if frames:
            data.append(frames[0])
            labels.append(row['label'])
    return np.array(data), np.array(labels)

# Data Preprocessing
train_images, train_labels = dataframe_to_dataset(MSASL_train)
test_images, test_labels = dataframe_to_dataset(MSASL_test)
val_images, val_labels = dataframe_to_dataset(MSASL_val)

train_images = train_images / 255.0
test_images = test_images / 255.0
val_images = val_images / 255.0

# Model Architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(IMAGE_HEIGHT, IMAGE_WIDTH, 3)), 
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(NUM_CLASSES, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the Model
model.fit(train_images, train_labels, epochs=EPOCHS, validation_data=(val_images, val_labels))

# Evaluate the Model
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_acc}")


  0%|                                                                        | 3/16054 [00:21<28:27:49,  6.38s/it]

Failed to download video from www.youtube.com/watch?v=9FdHlMOnVjg. Reason: 9FdHlMOnVjg is a private video


  0%|                                                                        | 6/16054 [00:40<26:01:36,  5.84s/it]

Failed to download video from https://www.youtube.com/watch?v=1AyT77LqJzQ. Reason: 1AyT77LqJzQ is unavailable


  0%|                                                                        | 7/16054 [00:42<19:21:03,  4.34s/it]

...

 16%|███████████                                                        | 2638/16054 [8:37:25<15:36:13,  4.19s/it]

Failed to download video from https://www.youtube.com/watch?v=TUQUVlMupcs. Reason: TUQUVlMupcs is unavailable


 16%|███████████                                                        | 2639/16054 [8:37:26<12:04:23,  3.24s/it]

Failed to download video from https://www.youtube.com/watch?v=TUQUVlMupcs. Reason: TUQUVlMupcs is unavailable


 17%|██████████▊                                                      | 2681/16054 [9:56:20<658:53:30, 177.37s/it]